In [2]:
import requests
import pandas as pd
import csv  
import time
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

from utils import extractor_v2 as v2
from utils import extractor_v3 as v3
from datetime import datetime


In [3]:
now = datetime.now()
print('Executed on:', now.strftime("%Y-%m-%d %H:%M:%S") )

Executed on: 2023-06-19 17:10:57


### Aave V2 stETH market. Data extraction

In [4]:
#Extraction and preparing the data about Aave positions with stETH as collateral
df = v2.get_data()
df = v2.prepare_data(df)
df.sort_values(by='amount',ascending= False)

,user,collateral,debt,available_borrow,threshold,ltv,healthf,amount,ethdebt,collateral_steth_calc,diff_collateral,diff_debt
0,0x777777c9898d384f785ee44acfe945efdff5f3e0,"216,300.41","51,884.62",103981454977953499370879,8281,7206,3.45,"171,718.34","25,938.93","171,552.03",0.21,0.50
1,0xa976ea51b9ba3232706af125a92e32788dc08ddc,"93,027.99","52,022.37",15106627111949065188618,8277,7216,1.48,"61,010.94","45,064.67","60,951.86",0.34,0.13
2,0x9600a48ed0f931d0c422d574e3275a90d8b22745,"47,808.47","37,883.72",0,8300,7200,1.05,"47,854.82","37,883.72","47,808.47",0.00,0.00
3,0x171c53d55b1bcb725f660677d9e8bad7fd084282,"32,144.27","22,968.34",175530122016854802523,8300,7200,1.16,"32,175.31",0.00,"32,144.15",0.00,1.00
4,0x7c07f7abe10ce8e33dc6c5ad68fe033085256a84,"28,966.30","17,929.73",2926004763423410907455,8300,7200,1.34,"28,994.38","17,929.73","28,966.30",0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1394,0x219da023416209c8c6cfb47d9903353541a359dc,0.10,0.07,2548618052704162,8300,7200,1.19,0.10,0.07,0.10,0.00,0.00
1428,0x28d0f70fa8c6e9657ff1290616b5afcbafb70eac,0.10,0.01,61789561613105805,8300,7200,7.20,0.10,0.00,0.10,0.00,1.00
1446,0x17a11b39e739deea6f690fe03117527b28f3ece2,0.10,0.01,67622868532180598,8300,7200,14.40,0.10,0.00,0.10,0.00,1.00
1441,0x608f4a15f525008e5d1d81309fcd5b5b1211d289,0.10,0.04,32293939090967554,8300,7200,2.06,0.10,0.00,0.10,0.00,1.00


In [5]:
print(f'Aave v2 stETH collateral: {df.amount.sum():,.0f} stETH')

Aave v2 stETH collateral: 883,276 stETH


In [6]:
# position's selection for Bin1: with >=80% of collateral in stETH and >= 80% of debt in ETH
b1 = v2.get_bin1_eth_debt(df)

In [7]:
print(f'In Bin1 there is {b1.amount.sum():,.0f} stETH, ~ {100*b1.amount.sum()/df.amount.sum():,.2f}% of total stETH locked in AAVE v2')

In Bin1 there is 229,672 stETH, ~ 26.00% of total stETH locked in AAVE v2


In [8]:
df = pd.read_csv("data/steth_aave.csv")
df = df.query('amount > 1')
df

,Unnamed: 0,user,collateral,debt,available_borrow,threshold,ltv,healthf,amount,ethdebt,collateral_steth_calc,diff_collateral,diff_debt
0,0,0x777777c9898d384f785ee44acfe945efdff5f3e0,"216,300.41","51,884.62",103981454977953499370879,8281,7206,3.45,"171,718.34","25,938.93","171,552.03",0.21,0.50
1,1,0xa976ea51b9ba3232706af125a92e32788dc08ddc,"93,027.99","52,022.37",15106627111949065188618,8277,7216,1.48,"61,010.94","45,064.67","60,951.86",0.34,0.13
2,2,0x9600a48ed0f931d0c422d574e3275a90d8b22745,"47,808.47","37,883.72",0,8300,7200,1.05,"47,854.82","37,883.72","47,808.47",0.00,0.00
3,3,0x171c53d55b1bcb725f660677d9e8bad7fd084282,"32,144.27","22,968.34",175530122016854802523,8300,7200,1.16,"32,175.31",0.00,"32,144.15",0.00,1.00
4,4,0x7c07f7abe10ce8e33dc6c5ad68fe033085256a84,"28,966.30","17,929.73",2926004763423410907455,8300,7200,1.34,"28,994.38","17,929.73","28,966.30",0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,1043,0x16aa47b96e7d9e35118dd4a1652e83a86f69ad87,1.04,0.68,64120465223051379,8300,7200,1.26,1.04,0.68,1.04,0.00,0.00
972,1044,0x7a41ed25316047f96eae5ddbe9e5eadc42eb7bc5,1.04,0.42,327148566846872074,8300,7200,2.05,1.04,0.00,1.04,0.00,1.00
973,1045,0x0d3450a8110d7fc8adad757bbd976a0aec7501e0,2.05,1.01,315275868525102826,7456,6470,1.51,1.02,1.01,1.02,0.50,0.00
974,1046,0xe823057e70a0430a254c052dc190f42fc80cb610,15.13,6.84,5519461473041945774,8578,8173,1.90,1.10,0.00,1.10,0.93,1.00


In [9]:
# position's selection for Bin2: with >=80% of debt not ETH and for Subbin (b2_debt80_stables) with >=80% of debt in stablecoins
b2_debt80_stables = v2.get_subbin2_stablecoin_debt(df)

In [10]:
b2 = pd.read_csv("data/b2.csv")
print(f'In Bin2 there is {b2.amount.sum():,.0f} stETH, ~ {100*b2.amount.sum()/df.amount.sum():,.2f} % of total stETH locked in AAVE v2')
print(f'In SubBin2 (with debt in stablecoins) there is {b2_debt80_stables.amount.sum():,.0f} stETH, ~ {100*b2_debt80_stables.amount.sum()/b2.amount.sum():,.2f} % of Bin2(v2)')


In Bin2 there is 371,733 stETH, ~ 42.09 % of total stETH locked in AAVE v2
In SubBin2 (with debt in stablecoins) there is 313,569 stETH, ~ 84.35 % of Bin2(v2)


In [11]:
# Selection of all remaining positions for Bin3 
# and allocation for Subbin (b3_eth_wbtc_stable_80) positions with >=80% of collateral in stETH, ETH, WBTC and stablecoins
# and >=80% of debt in stablecoins & WBTC
#b1 = pd.read_csv("b1.csv")
b3_eth_wbtc_stable_80 = v2.get_subbin3_stables_wbtc_eth_80(df, b1, b2)

In [12]:
b3 = pd.read_csv("data/b3.csv")
print(f'In Bin3 there is {b3.amount.sum():,.0f} stETH, ~ {100*b3.amount.sum()/df.amount.sum():,.2f} % of total stETH locked in AAVE v2')
print(f'In SubBin3 (with collateral&debt in stETH, ETH, stablecoins, WBTC) there is {b3_eth_wbtc_stable_80.amount.sum():,.0f} stETH, ~ {100*b3_eth_wbtc_stable_80.amount.sum()/b3.amount.sum():,.2f} % of Bin3 (v2)')


In Bin3 there is 281,811 stETH, ~ 31.91 % of total stETH locked in AAVE v2
In SubBin3 (with collateral&debt in stETH, ETH, stablecoins, WBTC) there is 272,615 stETH, ~ 96.74 % of Bin3 (v2)


### Aave V3 wstETH market. Data extraction

In [13]:
#Extraction and preparing the data about Aave v3 positions with stETH as collateral
dfv3 = v3.get_data()
dfv3 = dfv3.query('collateral > 0 & debt > 0')
dfv3 = v3.prepare_data(dfv3)

In [14]:
print(f'Aave v3 wstETH collateral: {dfv3.amount.sum():,.0f} wstETH')

Aave v3 wstETH collateral: 299,296 wstETH


In [15]:
# position's selection for Bin1.1: with >=80% of collateral in stETH/rETH/cbETH and >= 80% of debt in ETH and e-mode
b1v3 = v3.get_bin1_eth_debt(dfv3)
print(f'In Bin1.1 there is {b1v3.amount.sum():,.0f} stETH, ~ {100*b1v3.amount.sum()/dfv3.amount.sum():,.2f} % of total stETH locked in AAVE v3')

In Bin1.1 there is 165,222 stETH, ~ 55.20 % of total stETH locked in AAVE v3


In [16]:
# position's selection for Bin1.2: with >=80% of collateral in stETH/rETH/cbETH and >= 80% of debt in ETH and without e-mode - normally very small amount
b12v3 = v3.get_bin12_eth_debt(dfv3)
print(f'In Bin1.2 there is {b12v3.amount.sum():,.0f} stETH, ~ {100*b12v3.amount.sum()/dfv3.amount.sum():,.2f} % of total stETH locked in AAVE v3')

In Bin1.2 there is 643 stETH, ~ 0.21 % of total stETH locked in AAVE v3


In [17]:
# position's selection for Bin2: with >=80% of debt not ETH and for Subbin (b2_debt80_stables) with >=80% of debt in stablecoins

b2v3_debt80_stables = v3.get_subbin2_stablecoin_debt(dfv3)

In [18]:
b2v3 = pd.read_csv("data/b2_v3.csv")
print(f'In Bin2 there is {b2v3.amount.sum():,.0f} wstETH, ~ {100*b2v3.amount.sum()/dfv3.amount.sum():,.2f} % of total wstETH locked in AAVE v3')
print(f'In SubBin2 (with debt in stablecoins) there is {b2v3_debt80_stables.amount.sum():,.0f} stETH, ~ {100*b2v3_debt80_stables.amount.sum()/b2v3.amount.sum():,.2f} % of Bin2 (v3)')


In Bin2 there is 131,323 wstETH, ~ 43.88 % of total wstETH locked in AAVE v3
In SubBin2 (with debt in stablecoins) there is 101,305 stETH, ~ 77.14 % of Bin2 (v3)


In [19]:
# Selection of all remaining positions for Bin3 
# and allocation for Subbin (b3_eth_wbtc_stable_80) positions with >=80% of collateral in stETH, ETH, rETH, cbETH, WBTC and stablecoins
# and >=80% of debt in stablecoins & WBTC

b3v3_eth_wbtc_stable_80 = v3.get_subbin3_stables_wbtc_eth_80(dfv3, b1v3, b12v3, b2v3)
b3v3 = pd.read_csv("data/b3_v3.csv")
print(f'In Bin3 there is {b3v3.amount.sum():,.0f} stETH, ~ {100*b3v3.amount.sum()/dfv3.amount.sum():,.2f} % of total stETH locked in AAVE v3')
print(f'In SubBin3 (with collateral&debt in stETH, ETH, stablecoins, WBTC) there is {b3v3_eth_wbtc_stable_80.amount.sum():,.0f} stETH, ~ {100*b3v3_eth_wbtc_stable_80.amount.sum()/b3v3.amount.sum():,.2f} % of Bin3')


In Bin3 there is 2,108 stETH, ~ 0.70 % of total stETH locked in AAVE v3
In SubBin3 (with collateral&debt in stETH, ETH, stablecoins, WBTC) there is 2,106 stETH, ~ 99.92 % of Bin3
